In [1]:
%pwd

'/home/jovyan/workspace/Text-Summarization/research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/home/jovyan/workspace/Text-Summarization'

## Entities 

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strateg: str
    eval_steps: int 
    save_steps: float
    gradient_accumulation_steps: int
    


## configurtions manager in src/config

In [6]:
from src.textSummarization.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarization.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager():
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        print("Configuration Manager Initiated")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            data_path= Path(training.data_path),
            model_ckpt=Path(training.model_ckpt),
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strateg=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=float(params.save_steps),
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        return training_config


## Components

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

#### we are using the test dataset for training, whih is fatal and wrong. 
#### yet I am doing it anyhow to save time in prototyping. 
#### Dont forget to correct this in the actual code component and pipeline in src. 
#### use 'train', instead of 'test' in trainer in train(). 

In [9]:
class ModelTrainer():
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        seq2seq_data_collator = DataCollatorForSeq2Seq(
            tokenizer=tokenizer,
            model=model_pegasus
        )
        
        # loading data
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            save_steps=self.config.save_steps,
            evaluation_strategy=self.config.evaluation_strateg,
            eval_steps=self.config.eval_steps,
            logging_steps=self.config.logging_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            warmup_steps=self.config.warmup_steps,
            weight_decay=self.config.weight_decay
        )
        
        trainer = Trainer(
            model=model_pegasus,
            args=training_args,
            tokenizer=tokenizer,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"],
            data_collator=seq2seq_data_collator
        )
        
        trainer.train()
        
        # save the model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        
        # save the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))
        


## Pipeline

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_training_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e


Configuration Manager Initiated
[2023-10-23 20:33:52,896: INFO: common] yaml file: config/config.yaml loaded successfully
[2023-10-23 20:33:52,899: INFO: common] yaml file: params.yaml loaded successfully
[2023-10-23 20:33:52,900: INFO: common] directory: artifacts created successfully
[2023-10-23 20:33:52,901: INFO: common] directory: artifacts/model_trainer created successfully


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
